## This notebook will investigate right of way for some specific networks

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
sys.path.append('../src_variable')
import network as nw
import road as rd
import junction as jn
import traffic_lights as tl
import torch
import openGLUtils as glutils
import openGLUtilsGif as glgif
import FV_schemes as fv

### 2-1 junction

We consider the simplest junction in which there is the need for a priority parameter.
In this case it is only necessary with one priority parameter.

The specific network has two incoming roads and one outgoing road. There is only one outgoing road, and hence no crossing connections. Assume that road one has priority over road 2.

The distribution matrix takes the form
$$
A = 
\begin{bmatrix}
1 \\
1
\end{bmatrix}.
$$

For all cases we will assume the roads have lengths of 50 meters.

In [2]:
# Configuration of the network
distribution = [[1.0], [1.0]]
priorities = [[1], [2]]
crossing_connections =  [[[]],
                        [[]]]
L = 50
N = 5

#### Case 1:
Two equivalent roads with speed limits 50 km/h leading into a road with speed limit 50km/h. Almost no traffic in the beginning, but the combined influx into the two roads is bigger than the capacity of the outgoing road, so that there will arise some congestion.

For the first case we have no traffic lights.

In [3]:
# Creating the network
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.1, inflow = 0.3, id = "1_fw")
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, -1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.1, inflow = 0.3, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.0, id = "3_fw")

entering = [0,1]
leaving = [2]

junction = jn.Junction([road1, road2, road3], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 50
network = nw.RoadNetwork([road1, road2, road3], [junction], T)


In [4]:
densities, _, _, _ = network.solve_cons_law()

In [5]:
# Comment out to create the gif - note this will crash the kernel
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '2-1-row.gif')

#### Case 1:
Two equivalent roads with speed limits 50 km/h leading into a road with speed limit 50km/h. Here there is much traffic in the beginning, but the influx to the roads is less than the capacity of the outgoing road. Expected behaviour: traffic on road 1 should clear out before the traffic on the second road.

We still have no traffic lights.

In [6]:
# Creating the network
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, inflow = 0.1, id = "1_fw")
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, -1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, inflow = 0.1, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.0, id = "3_fw")

entering = [0,1]
leaving = [2]

junction = jn.Junction([road1, road2, road3], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 100
network = nw.RoadNetwork([road1, road2, road3], [junction], T)


In [7]:
densities, _, _, _ = network.solve_cons_law()

In [8]:
# Comment out to create the gif - note this will crash the kernel
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '2-1-row_2.gif')

### 2-2 junction

This is the smallest junction in which there may be crossing connections. Order the roads 1, 2, 3, 4, with 1 and 2 being the incoming roads, and 3 and 4 being the outgoing roads. Assume that road 1 has right of way. That means that it has priority going in to the two outgoing roads. In addition, road 2 crosses connection 1->3 when going to outgoing road 4. Hence, an upper bound needs to be calculated. Since there are two outgoing roads, we need a distribution matrix A. For the first case, assume that A takes the form
$$
A = 
\begin{bmatrix}
1 & 0\\
0 & 1
\end{bmatrix}
$$
that is, all traffic on road 1 continues to road 3 and all traffic on road 2 goes to road 4.


In [9]:
# Configuration of the network
distribution = [[1.0, 0.0], [0.0, 1.0]]
priorities = [[1, 1], [2, 2]]
crossing_connections =  [[[], []],
                        [[], [(0,0)]]]
L = 50
N = 5

Case 1:

All roads have the same maximum densities, and the same speed limits of 50 km/h. We assume that there are no traffic lights. Assume there is quite a lot of traffic on both roads, and not that high inflow of traffic.

In [10]:
# Creating the network
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, inflow = 0.3, id = "1_fw")
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.0, id = "3_fw")
road4 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.0, id = "4_fw")

entering = [0,1]
leaving = [2, 3]

junction = jn.Junction([road1, road2, road3, road4], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 150
network = nw.RoadNetwork([road1, road2, road3, road4], [junction], T)

In [11]:
for road in network.roads:
    road.update_index(0.0)

In [12]:
densities, queues, _, _ = network.solve_cons_law()

In [13]:
# dsfd

In [14]:
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '2-2-row.gif')

### 4-4 junction
This is the largest junction in the simulation, and we would therefore like to investigate some of the phenoma arising in this junction. Each road in the simulation is unidirectional, meaning that to be able to model a bi-directional road, we model it as two individual uni-directional roads. The junction we are considering is really only a 2-2 junction where each road is bi-directional, but for us we will model them as a 4-4 junction. We order the roads as following:
- road 1 goes from left to right into the junction
- road 2 goes from left to right out from the junction
- road 3 goes from right to left into the junction
- road 4 goes from rigth to left out from the junction
- road 5 goes from top to bottom into the junction
- road 6 goes from top to bottom out from the junction
- road 7 goes from bottom to top into the junction
- road 8 goes from bottom to top out from the junction

We will assume that the road going from left to right (and opposite) has right of way. We will also assume that most of the traffic will keep going, and that some of it will turn left or right. In addition to roads 1,2,3 and 4 having right of way, we will also assume a right hand rule where applicable. We summarize the distribution of traffic in the distribution matrix A:
$$
A = 
\begin{bmatrix}
0.8 & 0.0 & 0.1 & 0.1\\
0.0 & 0.8 & 0.1 & 0.1\\
0.25 & 0.25 & 0.5 & 0.0\\
0.25 & 0.25 & 0.0 & 0.5
\end{bmatrix}.
$$ 
We assume that u-turns are not allowed, which is why some of the terms are equal to zero.

Now for the crossing connections. Traffic from road 1 crosses no lanes when going to roads 2 and 6. However, when going to road 8, traffic from road 3 to road 4 is being crossed.
Traffic from road 3 crosses no lanes when going to road 4 and road 8. However, when going to road 6, traffic from road 1 to road 2 is being crossed.
Traffic from road 5 crosses no lanes when going to road 4. Traffic going to road 2 crosses traffic from road 3 to road 4, and traffic going to road 6 crosses traffic from road 3 to road 4, traffic from 1 to 2 and traffic from road 1 to road 8.
Traffic from road 7 crosses no lanes when going to road 2. Traffic  going to lane 4 crosses traffic from road 1 to road 2, and traffic going to road 8 crosses traffic from road 1 to road 2, road 3 to road 4 and road 3 to road 6. 
We summarize the crossing connections in the matrix $C$:
$$
C = 
\begin{bmatrix}
. & . & . & [(1,1)]\\
. & . & [(0,0)] & .\\
[(1,1)] & . & [(0,0),(1,1),(0,3)] & .\\
. & [(0,0)] & . & [(0,0), (1,1), (1,2)]
\end{bmatrix}
$$
We also need to specify the prirorities of the incoming roads for each outgoing road. This we summarize in the matrix $P$:
$$
P = 
\begin{bmatrix}
1 & 0 & 1 & 2\\
0 & 1 & 2 & 1\\
3 & 2 & 3 & 0\\
2 & 3 & 0 & 3
\end{bmatrix},
$$
here elements equal to zero indicate that the edge is not allowed. The positions of these elements should match with the zero elements of the distribution matrix.
We still assume that there are no traffic lights in the junction.

---

% (0,0) : 1 -> 2 (0,1) : 1 -> 4 (1,0) : 3 -> 2 (2,2) : 5 -> 6

### Test case 1

In [15]:
# Configuration of the network
# distribution = [[0.9, 0.0, 0.05, 0.05],
#                 [0.0, 0.9, 0.05, 0.05],
#                 [0.25, 0.25, 0.5, 0.0],
#                 [0.25, 0.25, 0.0, 0.5]]
distribution = [[0.9, 0.0, 0.1, 0.0],
                [0.0, 0.9, 0.0, 0.1],
                [0.05, 0.05, 0.9, 0.0],
                [0.05, 0.05, 0.0, 0.9]]
priorities = [[1, 0, 1, 2],
              [0, 1, 2, 1],
              [3, 2, 3, 0],
              [2, 3, 0, 3]]
crossing_connections =  [[[], [], [], [(1,1)]],
                         [[], [], [(0,0)], []],
                         [[(1,1)], [], [(0,0),(1,1),(0,3)], []],
                         [[], [(0,0)], [], [(0,0),(1,1),(1,2)]]]

L = 50
N = 5

In [16]:
torch.autograd.set_detect_anomaly(False)

In [17]:
# Creating the network
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (-0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, inflow = 0.3, id = "1_fw")
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0.1, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (1, 0), right_pos = (0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, inflow = 0.3, id = "3_bw")
road4 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-0.1, 0), right_pos = (-1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "4_bw")
road5 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, inflow = 0.12, id = "5_fw")
road6 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -0.1), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "6_fw")
road7 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -1), right_pos = (0, -0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, inflow = 0.12, id = "7_bw")
road8 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0.1), right_pos = (0, 1),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "8_bw")

entering = [0, 2, 4, 6]
leaving = [1, 3, 5, 7]

junction = jn.Junction([road1, road2, road3, road4, road5, road6, road7, road8], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 100
network = nw.RoadNetwork([road1, road2, road3, road4, road5, road6, road7, road8], [junction], T)

In [18]:
densities, queues, _, _ = network.solve_cons_law()

In [19]:
times = densities[0].keys()
objective = torch.tensor(0)
for i in range(len(densities)):
    for t in times:
        objective = objective + torch.sum(densities[i][t])

In [20]:
objective.backward()

Checking if autograd still works

In [21]:
# print(queues)
glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '4-4-test-big.gif')
# Check that equilibrium is correct...

End of simulation reached!
Saving GIF as: 4-4-test-big.gif


: 

#### Test case 2

In [ ]:
distribution = [[0.7, 0.0, 0.15, 0.15],
                [0.0, 0.7, 0.15, 0.15],
                [0.3, 0.3, 0.4, 0.0],
                [0.3, 0.3, 0.0, 0.4]]
priorities = [[1, 0, 1, 2],
              [0, 1, 2, 1],
              [3, 2, 3, 0],
              [2, 3, 0, 3]]
crossing_connections =  [[[], [], [], [(1,1)]],
                         [[], [], [(0,0)], []],
                         [[(1,1)], [], [(0,0),(1,1),(0,3)], []],
                         [[], [(0,0)], [], [(0,0),(1,1),(1,2)]]]

L = 50
N = 5

In [ ]:
# Creating the network
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (-0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, inflow = 0.3, id = "1_fw")
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0.1, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (1, 0), right_pos = (0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, inflow = 0.3, id = "3_bw")
road4 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-0.1, 0), right_pos = (-1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "4_bw")
road5 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, inflow = 0.12, id = "5_fw")
road6 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -0.1), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "6_fw")
road7 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -1), right_pos = (0, -0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, inflow = 0.12, id = "7_bw")
road8 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0.1), right_pos = (0, 1),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "8_bw")

entering = [0, 2, 4, 6]
leaving = [1, 3, 5, 7]

junction = jn.Junction([road1, road2, road3, road4, road5, road6, road7, road8], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 50
network = nw.RoadNetwork([road1, road2, road3, road4, road5, road6, road7, road8], [junction], T)

In [ ]:
densities, queues, _, _ = network.solve_cons_law()

In [ ]:
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '4-4-test2.gif')

#### Test case 3

In [ ]:
distribution = [[1.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0],
                [0.0, 0.0, 0.0, 1.0]]
priorities = [[1, 0, 1, 2],
              [0, 1, 2, 1],
              [3, 2, 3, 0],
              [2, 3, 0, 3]]
crossing_connections =  [[[], [], [], [(1,1)]],
                         [[], [], [(0,0)], []],
                         [[(1,1)], [], [(0,0),(1,1),(0,3)], []],
                         [[], [(0,0)], [], [(0,0),(1,1),(1,2)]]]

L = 50
N = 5

In [ ]:
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (-0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.15, id = "1_fw")
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0.1, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.1, inflow = 0.2, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (1, 0), right_pos = (0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.15, id = "3_bw")
road4 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-0.1, 0), right_pos = (-1, 0),
                initial = lambda x: torch.ones_like(x) * 0.1, inflow = 0.05, id = "4_bw")
road5 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0.1),
                initial = lambda x: torch.ones_like(x) * 0.6, inflow = 0.1, id = "5_fw")
road6 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -0.1), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.05, id = "6_fw")
road7 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -1), right_pos = (0, -0.1),
                initial = lambda x: torch.ones_like(x) * 0.6, inflow = 0.1, id = "7_bw")
road8 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0.1), right_pos = (0, 1),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.05, id = "8_bw")

entering = [0, 2, 4, 6]
leaving = [1, 3, 5, 7]

junction = jn.Junction([road1, road2, road3, road4, road5, road6, road7, road8], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 150
network = nw.RoadNetwork([road1, road2, road3, road4, road5, road6, road7, road8], [junction], T)

In [ ]:
densities, queues, _, _ = network.solve_cons_law()

In [ ]:
glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '4-4-test3.gif')

End of simulation reached!
Saving GIF as: 4-4-test3.gif


: 

In [ ]:
dfhf

In [ ]:
# Test end point of simulation - looks a bit strange
jnc = network.junctions[0]
rho_in = [road.rho[-road.pad] for road in jnc.road_in]
gamma_in = [road.gamma[road.idx] for road in jnc.road_in]
max_flux_in = [fv.fmax(gamma) for gamma in gamma_in]
max_dens_in = torch.tensor([road.max_dens for road in jnc.road_in])
rho_out = [road.rho[road.pad-1] for road in jnc.road_out]
gamma_out = [road.gamma[road.idx] for road in jnc.road_out]
max_dens_out = torch.tensor([road.max_dens for road in jnc.road_out])

In [ ]:
print("rho_in", rho_in)
print("gamma_in", gamma_in)
print("max_flux_in", max_flux_in)
print("max_dens_in", max_dens_in)
print("rho_out", rho_out)
print("gamma_out", gamma_out)
print("max_dens_out", max_dens_out)

rho_in [tensor(0.3000, grad_fn=<SelectBackward0>), tensor(0.7738, grad_fn=<SelectBackward0>), tensor(0.4750, grad_fn=<SelectBackward0>), tensor(0.8493, grad_fn=<SelectBackward0>)]
gamma_in [tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>)]
max_flux_in [tensor(0.0694, grad_fn=<DifferentiableGraphBackward>), tensor(0.0694, grad_fn=<DifferentiableGraphBackward>), tensor(0.0694, grad_fn=<DifferentiableGraphBackward>), tensor(0.0694, grad_fn=<DifferentiableGraphBackward>)]
max_dens_in tensor([1, 1, 1, 1])
rho_out [tensor(0.2831, grad_fn=<SelectBackward0>), tensor(0.1266, grad_fn=<SelectBackward0>), tensor(0.2154, grad_fn=<SelectBackward0>), tensor(0.2154, grad_fn=<SelectBackward0>)]
gamma_out [tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>)]
max_dens_out tensor([1, 1, 1, 1]

In [ ]:
n = len(jnc.entering)
m = len(jnc.leaving)
activation = jnc.calculate_activation(n, m, 60)
print(activation)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [ ]:
demand = jnc.calculate_demand(rho_in, gamma_in, activation, max_dens_in, n, m)
print(demand)
capacities = [max_dens_out[j] * fv.S(rho_out[j].clone(),  gamma_out[j]) for j in range(m)]
print(capacities)

tensor([[0.0408, 0.0000, 0.0088, 0.0088],
        [0.0000, 0.0486, 0.0104, 0.0104],
        [0.0208, 0.0208, 0.0277, 0.0000],
        [0.0208, 0.0208, 0.0000, 0.0278]], grad_fn=<CopySlices>)
[tensor(0.0694, grad_fn=<MulBackward0>), tensor(0.0694, grad_fn=<MulBackward0>), tensor(0.0694, grad_fn=<MulBackward0>), tensor(0.0694, grad_fn=<MulBackward0>)]


In [ ]:
priority_list = priorities
priority_params = [[0 for _ in range(len(leaving))] for _ in range(len(entering))]

for j in range(m):
    # Go through each of the outgoing roads and assign the supply to the incoming roads
    non_zero = [i for i in range(n) if priority_list[i][j] != 0]
    print(f"Non-zero prirorities leading into road {j}: {non_zero}")
    if len(non_zero) == 0:
        # No legal edges into road j -> This should never happen, maybe throw an error
        continue

    nz_priorities = [priorities[k][j] for k in non_zero]
    sorted_indexes = [x for _, x in sorted(zip(nz_priorities, non_zero))]

    if len(sorted_indexes) > 1:
        # More than one one legal edge into road j -> need prirority parameters
        density_in = distribution[sorted_indexes[0]][j] * rho_in[sorted_indexes[0]]
        priority_params[sorted_indexes[0]][j] = jn.priority_fnc(density_in)

        for i, idx in enumerate(sorted_indexes[1:-1]):
            # Go through the indexes in decreasing order
            # Don't need the last index -> set to 1 - sum(previous)
            density_in = distribution[idx][j] * rho_in[idx]
            nu_factor = jn.priority_fnc(density_in)
            priority_params[idx][j] = (1 - sum([priority_params[l][j] for l in sorted_indexes[:i+1]])) * nu_factor

        priority_params[sorted_indexes[-1]][j] = 1 - sum([priority_params[l][j] for l in sorted_indexes[:-1]])
    
    elif len(sorted_indexes) == 1:
        # Only one legal edge into road j -> set priority parameter to 1
        priority_params[sorted_indexes[0]][j] = 1

priority_params


Non-zero prirorities leading into road 0: [0, 2, 3]
Non-zero prirorities leading into road 1: [1, 2, 3]
Non-zero prirorities leading into road 2: [0, 1, 2]
Non-zero prirorities leading into road 3: [0, 1, 3]


[[tensor(0.7289, grad_fn=<AddBackward0>),
  0,
  tensor(0.6241, grad_fn=<AddBackward0>),
  tensor(0.2078, grad_fn=<MulBackward0>)],
 [0,
  tensor(0.8946, grad_fn=<AddBackward0>),
  tensor(0.2508, grad_fn=<MulBackward0>),
  tensor(0.6671, grad_fn=<AddBackward0>)],
 [tensor(0.0655, grad_fn=<RsubBackward1>),
  tensor(0.0721, grad_fn=<MulBackward0>),
  tensor(0.1252, grad_fn=<RsubBackward1>),
  0],
 [tensor(0.2056, grad_fn=<MulBackward0>),
  tensor(0.0333, grad_fn=<RsubBackward1>),
  0,
  tensor(0.1252, grad_fn=<RsubBackward1>)]]

In [ ]:
priorities = jnc.calculate_priority_params(rho_in, n, m)
for j in range(4):
    print(sum([priorities[i][j] for i in range(4)]))

tensor(1., grad_fn=<AddBackward0>)
tensor(1., grad_fn=<AddBackward0>)
tensor(1., grad_fn=<AddBackward0>)
tensor(1., grad_fn=<AddBackward0>)


In [ ]:
fluxes_in, fluxes_out = jnc.calculate_fluxes(demand, capacities, priorities, n, m, 
                                                      max_flux_in, max_dens_in, max_dens_out)
print(fluxes_in)
print(fluxes_out)


[tensor(0.0583, grad_fn=<DivBackward0>), tensor(0.0487, grad_fn=<DivBackward0>), tensor(0.0384, grad_fn=<DivBackward0>), tensor(0.0357, grad_fn=<DivBackward0>)]
[tensor(0.0564, grad_fn=<DivBackward0>), tensor(0.0308, grad_fn=<DivBackward0>), tensor(0.0469, grad_fn=<DivBackward0>), tensor(0.0469, grad_fn=<DivBackward0>)]


In [ ]:
0.0486 + 2*0.0104

0.06939999999999999

# Everything below is just for testing during development
---
---
---
---

In [ ]:
# Checking flux distribution manually
jnc = network.junctions[0]
rho_in = [road.rho[-road.pad] for road in jnc.road_in]
gamma_in = [road.gamma[road.idx] for road in jnc.road_in]
max_flux_in = [fv.fmax(gamma) for gamma in gamma_in]
max_dens_in = torch.tensor([road.max_dens for road in jnc.road_in])
rho_out = [road.rho[road.pad-1] for road in jnc.road_out]
gamma_out = [road.gamma[road.idx] for road in jnc.road_out]
max_dens_out = torch.tensor([road.max_dens for road in jnc.road_out])

In [ ]:
print("rho_in", rho_in)
print("gamma_in", gamma_in)
print("max_flux_in", max_flux_in)
print("max_dens_in", max_dens_in)
print("rho_out", rho_out)
print("gamma_out", gamma_out)
print("max_dens_out", max_dens_out)

rho_in [tensor(0.3000, grad_fn=<SelectBackward0>), tensor(0.7738, grad_fn=<SelectBackward0>), tensor(0.4750, grad_fn=<SelectBackward0>), tensor(0.8493, grad_fn=<SelectBackward0>)]
gamma_in [tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>)]
max_flux_in [tensor(0.0694, grad_fn=<DifferentiableGraphBackward>), tensor(0.0694, grad_fn=<DifferentiableGraphBackward>), tensor(0.0694, grad_fn=<DifferentiableGraphBackward>), tensor(0.0694, grad_fn=<DifferentiableGraphBackward>)]
max_dens_in tensor([1, 1, 1, 1])
rho_out [tensor(0.2831, grad_fn=<SelectBackward0>), tensor(0.1266, grad_fn=<SelectBackward0>), tensor(0.2154, grad_fn=<SelectBackward0>), tensor(0.2154, grad_fn=<SelectBackward0>)]
gamma_out [tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>)]
max_dens_out tensor([1, 1, 1, 1]

In [ ]:
n = len(jnc.entering)
m = len(jnc.leaving)
activation = jnc.calculate_activation(n, m, 60)
print(activation)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [ ]:
fv.flux(torch.tensor(0.2), torch.tensor(0.2778))

tensor(0.0444)

In [ ]:
demand = jnc.calculate_demand(rho_in, gamma_in, activation, max_dens_in, n, m)
print(demand)
capacities = [max_dens_out[j] * fv.S(rho_out[j].clone(),  gamma_out[j]) for j in range(m)]
print(capacities)
# Demand and capacities seem reasonable

tensor([[0.0408, 0.0000, 0.0088, 0.0088],
        [0.0000, 0.0486, 0.0104, 0.0104],
        [0.0208, 0.0208, 0.0277, 0.0000],
        [0.0208, 0.0208, 0.0000, 0.0278]], grad_fn=<CopySlices>)
[tensor(0.0694, grad_fn=<MulBackward0>), tensor(0.0694, grad_fn=<MulBackward0>), tensor(0.0694, grad_fn=<MulBackward0>), tensor(0.0694, grad_fn=<MulBackward0>)]


Non-zero prirorities leading into road 0: [0, 1, 2]
Non-zero prirorities leading into road 1: [0, 1, 2]
Non-zero prirorities leading into road 2: [0, 1, 2]
Non-zero prirorities leading into road 3: [0, 1, 2]


In [ ]:
priorities = jnc.calculate_priority_params(rho_in, n, m)
print(priorities)
# Probably okay too

[[tensor(0.7289, grad_fn=<AddBackward0>), tensor(0.6000, grad_fn=<AddBackward0>), tensor(0.6241, grad_fn=<AddBackward0>), tensor(0.6241, grad_fn=<AddBackward0>)], [tensor(0.1627, grad_fn=<MulBackward0>), tensor(0.3578, grad_fn=<MulBackward0>), tensor(0.2508, grad_fn=<MulBackward0>), tensor(0.2508, grad_fn=<MulBackward0>)], [tensor(0.1084, grad_fn=<RsubBackward1>), tensor(0.0422, grad_fn=<RsubBackward1>), tensor(0.1252, grad_fn=<RsubBackward1>), tensor(0.1252, grad_fn=<RsubBackward1>)], [0, 0, 0, 0]]


In [ ]:
actual_fluxes = torch.zeros((n, m))
assigned_fluxes = []
upper_bounds = demand.clone()

for n_crossing in range(3 + 1):
    for i in range(n):
        for j in range(m):
            if len(crossing_connections[i][j]) == n_crossing and (i,j) not in assigned_fluxes:
                # Connection has not been assigned a flux, and it has the correct number of crossing connections
                if len(crossing_connections[i][j]) > 0:
                    # Update upper bound
                    upper_bounds[i,j] = jnc.calculate_upper_bound(actual_fluxes, 
                                                                    crossing_connections[i][j],
                                                                    i, max_flux_in, demand[i,j])
                    print(f"Upper bound on connection ({i},{j}) calculated as {upper_bounds[i,j]}")
                # Calculate actual flux
                # upper_bound[i,j] should be less or equal to demand[i,j] and so
                # it should not be necessary to include upper_bounds[i,j] below
                # demand_sum = torch.tensor(0.0)
                upper_bound_sum = torch.tensor(0.0)
                for l in range(n):
                    if l != i:
                        # demand_sum += demand[l,j]
                        # upper_bound_sum += upper_bounds[l,j]
                        upper_bound_sum += torch.min(upper_bounds[l,j].clone(), demand[l,j].clone())
                # interior_max = torch.max(capacities[j] - demand_sum,
                #                                 capacities[j] - upper_bound_sum)
                interior_max = capacities[j] - upper_bound_sum
                supply_max = torch.max(priorities[i][j]*capacities[j], interior_max)
                print(f"Maximum supply on connection ({i}, {j}): {priorities[i][j]*capacities[j]}")
                print(f"Maximum supply on connection ({i}, {j}): {supply_max}")

                demand_max = torch.min(upper_bounds[i,j].clone(), demand[i,j].clone())
                actual_fluxes[i,j] = torch.min(demand_max, supply_max.clone())
                
                # actual_fluxes[i,j] = torch.min(torch.min(upper_bounds[i,j], demand[i,j]), 
                #                             torch.max(priorities[i][j]*capacities[j],
                #                                 torch.max(
                #                                 capacities[j] - demand_sum,
                #                                 capacities[j] - upper_bound_sum
                #                                 )))
                    

                assigned_fluxes.append((i,j))

Maximum supply on connection (0, 0): 0.05061863735318184
Maximum supply on connection (0, 0): 0.05061863735318184
Maximum supply on connection (0, 1): 0.0416666716337204
Maximum supply on connection (0, 1): 0.0416666716337204
Maximum supply on connection (0, 2): 0.043337833136320114
Maximum supply on connection (0, 2): 0.043337833136320114
Maximum supply on connection (1, 0): 0.011295486241579056
Maximum supply on connection (1, 0): 0.011295486241579056
Maximum supply on connection (1, 1): 0.024850208312273026
Maximum supply on connection (1, 1): 0.02782968059182167
Maximum supply on connection (1, 3): 0.017415549606084824
Maximum supply on connection (1, 3): 0.03291592001914978
Maximum supply on connection (2, 1): 0.002927569206804037
Maximum supply on connection (2, 1): 0.002927569206804037
Maximum supply on connection (2, 3): 0.008691069670021534
Maximum supply on connection (2, 3): 0.02249925583600998
Maximum supply on connection (3, 0): 0.0
Maximum supply on connection (3, 0): 0.0

In [ ]:
fluxes_in, fluxes_out = jnc.calculate_fluxes(demand, capacities, priorities, n, m, 
                                                      max_flux_in, max_dens_in, max_dens_out)
print(fluxes_in)
print(fluxes_out)

# All flux entering junction is allowed to enter...

[tensor(0.0583, grad_fn=<DivBackward0>), tensor(0.0487, grad_fn=<DivBackward0>), tensor(0.0384, grad_fn=<DivBackward0>), tensor(0.0357, grad_fn=<DivBackward0>)]
[tensor(0.0564, grad_fn=<DivBackward0>), tensor(0.0308, grad_fn=<DivBackward0>), tensor(0.0469, grad_fn=<DivBackward0>), tensor(0.0469, grad_fn=<DivBackward0>)]


In [ ]:
actual_fluxes = torch.zeros((n, m))

assigned_fluxes = []
upper_bounds = demand.clone()

for n_crossing in range(jnc.max_crossing_connections + 1):
            for i in range(n):
                for j in range(m):
                    if len(jnc.crossing_connections[i][j]) == n_crossing and (i,j) not in assigned_fluxes:
                        # Connection has not been assigned a flux, and it has the correct number of crossing connections
                        if len(jnc.crossing_connections[i][j]) > 0:
                            # Update upper bound
                            print("Upper bound calculated:")
                            print("actual_fluxes", actual_fluxes)
                            print(jnc.calculate_upper_bound(actual_fluxes, 
                                                        jnc.crossing_connections[i][j],
                                                        i, j, n, m, max_flux_in, demand[i,j]))
                        # Calculate actual flux
                        # upper_bound[i,j] should be less or equal to demand[i,j] and so
                        # it should not be necessary to include upper_bounds[i,j] below
                        print("New flux is being calculated:")
                        print(f"Upper bound: {upper_bounds[i,j]}")
                        print(f"Demand: {demand[i,j]}")
                        print(f"Distributed priority: {priorities[i][j]*capacities[j]}")
                        print(f"Capcities - deman: {capacities[j] - torch.sum(demand[:,j])}")
                        print(f"Capcities - upper bounds: {capacities[j] - torch.sum(upper_bounds[:,j])}")
                        demand_sum = torch.tensor(0.0)
                        upper_bound_sum = torch.tensor(0.0)
                        for l in range(n):
                            if l != i:
                                demand_sum += demand[l,j]
                                upper_bound_sum += upper_bounds[l,j]

                        print(torch.min(torch.min(upper_bounds[i,j], demand[i,j]), 
                                                    torch.max(priorities[i][j]*capacities[j],
                                                        torch.max(
                                                        capacities[j] - demand_sum,
                                                        capacities[j] - upper_bound_sum
                                                        ))))
                            
                        print()
                        assigned_fluxes.append((i,j))

New flux is being calculated:
Upper bound: 0.04083681106567383
Demand: 0.04083681106567383
Distributed priority: 0.05061863735318184
Capcities - deman: -0.013007134199142456
Capcities - upper bounds: -0.013007134199142456
tensor(0.0408, grad_fn=<MinimumBackward0>)

New flux is being calculated:
Upper bound: 0.0
Demand: 0.0
Distributed priority: 0.0416666716337204
Capcities - deman: -0.020781435072422028
Capcities - upper bounds: -0.020781435072422028
tensor(0., grad_fn=<MinimumBackward0>)

New flux is being calculated:
Upper bound: 0.00875074602663517
Demand: 0.00875074602663517
Distributed priority: 0.043337833136320114
Capcities - deman: 0.022568456828594208
Capcities - upper bounds: 0.022568456828594208
tensor(0.0088, grad_fn=<MinimumBackward0>)

New flux is being calculated:
Upper bound: 0.0
Demand: 0.0
Distributed priority: 0.011295486241579056
Capcities - deman: -0.013007134199142456
Capcities - upper bounds: -0.013007134199142456
tensor(0., grad_fn=<MinimumBackward0>)

New flux 

TypeError: Junction.calculate_upper_bound() takes from 6 to 7 positional arguments but 9 were given

In [ ]:
glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '2-2-row.gif')

: 

In [ ]:
fdkjlsg

NameError: name 'fdkjlsg' is not defined

In [ ]:
# Stationary point (0.1, 0.9053, 0.2275) found
# Should this actually be a stationary point?

# Calculate fluxes in:
rho_in = [road.rho[-road.pad] for road in network.junctions[0].road_in]
gamma_in = [road.gamma[road.idx] for road in network.junctions[0].road_in]
max_flux_in = [fv.fmax(gamma) for gamma in gamma_in]
max_dens_in = torch.tensor([road.max_dens for road in network.junctions[0].road_in])
rho_out = [road.rho[road.pad-1] for road in network.junctions[0].road_out]
gamma_out = [road.gamma[road.idx] for road in network.junctions[0].road_out]
max_dens_out = torch.tensor([road.max_dens for road in network.junctions[0].road_out])

In [ ]:
# Quantities seem reasonable
print(rho_in)
print(gamma_in)
print(rho_out)
print(gamma_out)
print(max_dens_in)
print(max_dens_out)

[tensor(0.1000, grad_fn=<SelectBackward0>), tensor(0.9053, grad_fn=<SelectBackward0>)]
[tensor(0.2778, grad_fn=<DivBackward0>), tensor(0.2778, grad_fn=<DivBackward0>)]
[tensor(0.2275, grad_fn=<SelectBackward0>)]
[tensor(0.2778, grad_fn=<DivBackward0>)]
tensor([1, 1])
tensor([1])


In [ ]:
jnc = network.junctions[0]

In [ ]:
n = len(jnc.entering)
m = len(jnc.leaving)
activation = jnc.calculate_activation(n, m, 60)
print(activation)
# Activation seems reasonable

tensor([[1.],
        [1.]])


In [ ]:
demand = jnc.calculate_demand(rho_in, gamma_in, activation, max_dens_in, n, m)
print(demand)
capacities = [max_dens_out[j] * fv.S(rho_out[j].clone(),  gamma_out[j]) for j in range(m)]
print(capacities)
# Demand and capacities seem reasonable

tensor([[0.0250],
        [0.0694]], grad_fn=<CopySlices>)
[tensor(0.0694, grad_fn=<MulBackward0>)]


In [ ]:
priorities = jnc.calculate_priority_params(rho_in, n, m)
print(priorities)
# Probably okay too

[[tensor(0.6569, grad_fn=<AddBackward0>)], [tensor(0.3431, grad_fn=<RsubBackward1>)]]


In [ ]:
fluxes_in, fluxes_out = jnc.calculate_fluxes(demand, capacities, priorities, n, m, 
                                                      max_flux_in, max_dens_in, max_dens_out)
print(fluxes_in)
print(fluxes_out)
# -> Not all of the capacity is being used, why is this?

[tensor(0.0250, grad_fn=<DivBackward0>), tensor(0.0238, grad_fn=<DivBackward0>)]
[tensor(0.0488, grad_fn=<DivBackward0>)]


In [ ]:
demand.sum(dim=0, keepdim=True)

tensor([[0.0944]], grad_fn=<SumBackward1>)

In [ ]:
# Doing the steps manually
actual_fluxes = torch.zeros((n, m))

assigned_fluxes = []
upper_bounds = demand.clone()

for n_crossing in range(jnc.max_crossing_connections + 1):
            for i in range(n):
                for j in range(m):
                    if len(jnc.crossing_connections[i][j]) == n_crossing and (i,j) not in assigned_fluxes:
                        # Connection has not been assigned a flux, and it has the correct number of crossing connections
                        if len(jnc.crossing_connections[i][j]) > 0:
                            # Update upper bound
                            print("Upper bound calculated:")
                            print(jnc.calculate_upper_bound(actual_fluxes, 
                                                        jnc.crossing_connections[i][j],
                                                        i, j, n, m, max_flux_in, demand[i,j]))
                        # Calculate actual flux
                        # upper_bound[i,j] should be less or equal to demand[i,j] and so
                        # it should not be necessary to include upper_bounds[i,j] below
                        print("New flux is being calculated:")
                        print(f"Upper bound: {upper_bounds[i,j]}")
                        print(f"Demand: {demand[i,j]}")
                        print(f"Distributed priority: {priorities[i][j]*capacities[j]}")
                        print(f"Capcities - deman: {capacities[j] - torch.sum(demand[:,j])}")
                        print(f"Capcities - upper bounds: {capacities[j] - torch.sum(upper_bounds[:,j])}")
                        demand_sum = torch.tensor(0.0)
                        upper_bound_sum = torch.tensor(0.0)
                        for l in range(n):
                            if l != i:
                                demand_sum += demand[l,j]
                                upper_bound_sum += upper_bounds[l,j]

                        print(torch.min(torch.min(upper_bounds[i,j], demand[i,j]), 
                                                    torch.max(priorities[i][j]*capacities[j],
                                                        torch.max(
                                                        capacities[j] - demand_sum,
                                                        capacities[j] - upper_bound_sum
                                                        ))))
                            
                        print()
                        assigned_fluxes.append((i,j))

New flux is being calculated:
Upper bound: 0.02500000037252903
Demand: 0.02500000037252903
Distributed priority: 0.04562114551663399
Capcities - deman: -0.02499999850988388
Capcities - upper bounds: -0.02499999850988388
tensor(0.0250, grad_fn=<MinimumBackward0>)

New flux is being calculated:
Upper bound: 0.0694444477558136
Demand: 0.0694444477558136
Distributed priority: 0.02382330223917961
Capcities - deman: -0.02499999850988388
Capcities - upper bounds: -0.02499999850988388
tensor(0.0444, grad_fn=<MinimumBackward0>)

